# Evaluating Numerical Expressions

The goal of this project is to use the stack data structure to implement an algorithm that can evaluate complex numerical expressions.

## Importing the LinkedList and Stack

In [1]:
from linked_list import LinkedList

class Stack(LinkedList):
    def push(self, data):  # add data at top of tail
        self.append(data)
        
    def peek(self):        # return top element
        return self.tail.data
        
    def pop(self):         # remove top element
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret


## Implementing the tokenize function

In [2]:
def tokenize(expression):
    return expression.split()

tokenize("12 2 4 + / 21 *")

['12', '2', '4', '+', '/', '21', '*']

## Process operators in postfix evaluation

The functions are all the same, the only thing that changes is the operator used to calculate the `result` variable.

It is very important to perform the operation between the elements that was second to to and the top elements. If we do it the other way around we'll get the wrong result.

For example, in the `process_minus()` function we do:

result = second_to_top - top # Correct
and not

result = top - second_to_top # Wrong

In [3]:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)
    
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)
    
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)
    
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)
    
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

## Evaluating Postfix Expressions

Here are the steps we need to follow to implement the evaluate_postfix() function.

1. Initialize an empty stack.
2. Tokenize the expression using the tokenize() function.
3. For each token, do:

    A). If the token an operator, call the corresponding function to process it. For example, if we find a + we call the process_plus() function.
    
    B). Otherwise (the token is a number) and we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a float first.


4. Return the value that is left in the stack.

In [4]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            # The token is not an operator so it must be a number
            stack.push(float(token))
    return stack.pop()

## Test Implementation

When testing with other expressions we need to add spaces between at two tokens. For example `1 + 3` will work but `1+3` won't.

In [5]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for exp in expressions:
    print(evaluate_postfix(exp))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Precedence dictionary

The precedence dictionary is used to compare the precedence of two operators.

In [6]:
precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3
}

print(precedence["/"] < precedence["-"])
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

False
True
False
True


## Processing tokens in infix to postfix conversions

### Opening parenthesis
- Opening parentheses, `(`:
1. Push the token into the stack. It will be used later when we find a closing parenthesis.

In [7]:
def process_opening_parenthesis(stack):
    stack.push("(")

### Closing parenthesis

- Closing parentheses `)`:
1. While the top of the stack is not an opening parenthesis, (, pop the top element and append it to the postfix token list.
2. Pop the opening parentheses out of the stack at the end.

In [8]:
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        postfix.append(stack.pop())
    stack.pop()

### Operators

- Operator, `+`, `-`, `*`, `/` or `**`:
1. While the top of the stack is also an operator whose precedence is greater than or equal to this operator, pop the top element and append it to the `postfix` token list.
2. Push the current operator to the top of the stack.

The `Stack.peek()` method will cause an error if the stack is empty. Thus, in the while loop we also need to check that the stack is not empty.

In [9]:
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    stack.push(operator)

### Numbers

- Operand (any number):
1. Push the token into the the postfix token list.

In [10]:
def process_number(postfix, number):
    postfix.append(number)

### The Shunting-yard Algorithm

In [11]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)

## Evaluating Infix Expressions

In [12]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

In [16]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

In [17]:
for expr in expressions:
    print(evaluate(expr))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
